In [1]:
# Q1
'''the polynomial kernel is a kernel function commonly used with support vector machines (SVMs) and other kernelized models,
   that represents the similarity of vectors (training samples) in a feature space over polynomials of the original variables,
   allowing learning of non-linear models.
   the polynomial kernel looks not only at the given features of input samples to determine their similarity, but also 
   combinations of these. In the context of regression analysis, such combinations are known as interaction features. 
   The (implicit) feature space of a polynomial kernel is equivalent to that of polynomial regression, but without the 
   combinatorial blowup in the number of parameters to be learned.
   '''

'the polynomial kernel is a kernel function commonly used with support vector machines (SVMs) and other kernelized models,\n   that represents the similarity of vectors (training samples) in a feature space over polynomials of the original variables,\n   allowing learning of non-linear models.\n   the polynomial kernel looks not only at the given features of input samples to determine their similarity, but also \n   combinations of these. In the context of regression analysis, such combinations are known as interaction features. \n   The (implicit) feature space of a polynomial kernel is equivalent to that of polynomial regression, but without the \n   combinatorial blowup in the number of parameters to be learned.\n   '

In [2]:
# Q2
'''we implement an SVM with a polynomial kernel in Python using Scikit-learn:

Importing the necessary libraries.
import numpy as np.
from sklearn. datasets import load_digits.
from sklearn. model_selection import train_test_split.
from sklearn. svm import SVC.
'''

'we implement an SVM with a polynomial kernel in Python using Scikit-learn:\n\nImporting the necessary libraries.\nimport numpy as np.\nfrom sklearn. datasets import load_digits.\nfrom sklearn. model_selection import train_test_split.\nfrom sklearn. svm import SVC.\n'

In [3]:
# Q3
'''If epsilon is zero, we can expect overfitting
    Parameter ε controls the width of the ε-insensitive zone, used to fit the training data. The value of ε can affect the
    number of support vectors used to construct the regression function. The bigger ε, the fewer support vectors are selected.
    
    SVR has an additional tunable parameter ε (epsilon). The value of epsilon determines the width of the tube around the
    estimated function (hyperplane). Points that fall inside this tube are considered as correct predictions and are not
    penalized by the algorithm.'''

'If epsilon is zero, we can expect overfitting\n    Parameter ε controls the width of the ε-insensitive zone, used to fit the training data. The value of ε can affect the\n    number of support vectors used to construct the regression function. The bigger ε, the fewer support vectors are selected.\n    \n    SVR has an additional tunable parameter ε (epsilon). The value of epsilon determines the width of the tube around the\n    estimated function (hyperplane). Points that fall inside this tube are considered as correct predictions and are not\n    penalized by the algorithm.'

In [4]:
# Q4
'''The gamma parameters can be seen as the inverse of the radius of influence of samples selected by the model as support 
   vectors.
   The C parameter trades off correct classification of training examples against maximization of the decision function’s
   margin.
   '''


'The gamma parameters can be seen as the inverse of the radius of influence of samples selected by the model as support \n   vectors.\n   The C parameter trades off correct classification of training examples against maximization of the decision function’s\n   margin.\n   '

In [5]:
# Q5
import numpy as np
from matplotlib.colors import Normalize


class MidpointNormalize(Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target
X_2d = X[:, :2]
X_2d = X_2d[y > 0]
y_2d = y[y > 0]
y_2d -= 1

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_2d = scaler.fit_transform(X_2d)

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(X, y)

print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)

The best parameters are {'C': 1.0, 'gamma': 0.1} with a score of 0.97


In [8]:
C_2d_range = [1e-2, 1, 1e2]
gamma_2d_range = [1e-1, 1, 1e1]
classifiers = []
for C in C_2d_range:
    for gamma in gamma_2d_range:
        clf = SVC(C=C, gamma=gamma)
        clf.fit(X_2d, y_2d)
        classifiers.append((C, gamma, clf))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
xx, yy = np.meshgrid(np.linspace(-3, 3, 200), np.linspace(-3, 3, 200))
for k, (C, gamma, clf) in enumerate(classifiers):
    # evaluate decision function in a grid
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # visualize decision function for these parameters
    plt.subplot(len(C_2d_range), len(gamma_2d_range), k + 1)
    plt.title("gamma=10^%d, C=10^%d" % (np.log10(gamma), np.log10(C)), size="medium")

    # visualize parameter's effect on decision function
    plt.pcolormesh(xx, yy, -Z, cmap=plt.cm.RdBu)
    plt.scatter(X_2d[:, 0], X_2d[:, 1], c=y_2d, cmap=plt.cm.RdBu_r, edgecolors="k")
    plt.xticks(())
    plt.yticks(())
    plt.axis("tight")

scores = grid.cv_results_["mean_test_score"].reshape(len(C_range), len(gamma_range))

In [ ]:
plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=0.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(
    scores,
    interpolation="nearest",
    cmap=plt.cm.hot,
    norm=MidpointNormalize(vmin=0.2, midpoint=0.92),
)
plt.xlabel("gamma")
plt.ylabel("C")
plt.colorbar()
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.title("Validation accuracy")
plt.show()